<a href="https://colab.research.google.com/github/MariaMalycha/dw_matrix/blob/master/Day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install eli5

In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold

import eli5 
from eli5.sklearn import PermutationImportance
#dealing with dictionaries
from ast import literal_eval
from tqdm import tqdm_notebook


# Modeling

In [8]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [9]:
ls data

men-shoes.csv


In [10]:
df = pd.read_csv('data/men-shoes.csv', low_memory=False)
df.shape

(18280, 48)

In [11]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df.prices_amountmin.values
  
  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
df['brand_cat'] = df.brand.map(lambda x: str(x).lower()).factorize()[0]

In [14]:
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [16]:
model=RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [17]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict

  features = literal_eval(x.replace('\\"', '"'))
  #[{'key': 'Gender', 'value': ['Men']}
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()
    output_dict[key] = value
  
  return output_dict
  
df['features_parsed'] = df['features'].map(parse_features)

In [19]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))
len(keys)

476

In [20]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [21]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_what includes', 'feat_number of pockets', 'feat_heel height',
       'feat_motion control', 'feat_shoe width', 'feat_ean', 'feat_type 2',
       'feat_occasion', 'feat_country/ organization', 'feat_removable liner'],
      dtype='object', length=526)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False == df[ get_name_feat(key) ].isnull() ].shape[0] / df.shape[0] * 100

In [23]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
'''
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]
'''

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]


In [25]:
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [26]:
df['brand'] = df['brand'].map( lambda x: str(x).lower() )
df[ df.brand != df.feat_brand ][ ['brand', 'feat_brand'] ].shape

(9434, 2)

In [0]:
feats = ['']

In [28]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
feats = ['brand_cat']
run_model( feats, model)


(-57.40481944286989, 4.245719472026112)

In [0]:
#feats_cat = [x for x in df.columns if 'cat' in x]

feats_cat = [x for x in df.columns if '_cat' in x and x != 'feat_catalog']


In [54]:
feats = ['brand_cat' 
         ,'feat_style_cat'
         ,'feat_sport_cat'
         ,'feat_brand_cat' 
         ,'feat_gender_cat'  
         ,'feat_material_cat']
#feats += feats_search
#feats = list(set(feats))
model = RandomForestRegressor(max_depth=5, n_estimators=100)
result_benchmark = run_model( feats, model)
result_benchmark

(-57.03058835104404, 4.2599319514433125)

In [61]:
print(len(feats_cat))
feats_search = feats_cat[301:400]

477


In [97]:
feats = ['brand_cat' 
         ,'feat_age_cat'
          ,'feat_weight_cat'
         ,'feat_brand_cat' 
         ,'feat_gender_cat'  
         ,'feat_material_cat']


#feats += feats_search
feats = list(set(feats))
model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model( feats, model)
print(result_benchmark)
print(result)

(-57.03058835104404, 4.2599319514433125)
(-57.058160171730364, 4.2866771636108885)


In [84]:
X = df[ feats ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)
print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)


(-57.02158637270962, 4.299514552240644)


Weight,Feature
0.2613 ± 0.0087,brand_cat
0.1031 ± 0.0057,feat_material_cat
0.0290 ± 0.0012,feat_age_cat
0.0264 ± 0.0025,feat_gender_cat
0.0198 ± 0.0021,feat_brand_cat
0.0128 ± 0.0009,feat_weight_cat


In [98]:
df['brand'].value_counts(normalize=True)

nike                  0.097210
puma                  0.033315
ralph lauren          0.028775
vans                  0.021116
new balance           0.020295
                        ...   
deadman wonderland    0.000055
wings & horns         0.000055
john ashford          0.000055
wisconsin badgers     0.000055
zanzara               0.000055
Name: brand, Length: 1732, dtype: float64

In [101]:
df[ df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'condition': 'new with box', 'color': 'black'},
       {'material': 'synthetic'},
       {'style': 'basketball shoes', 'country/region of manufacture': 'china', 'condition': 'pre-owned'},
       {'sport': 'football', 'country/region of manufacture': 'china', 'condition': 'new with box', 'type': 'cleats'},
       {'primary color': 'blue', 'battery type': 'does not contain a battery', 'lens tint': 'grey', 'product in inches (l x w x h)': '6.0 x 2.0 x 2.0', 'assembled product dimensions (l x w x h)': '6.00 x 2.00 x 2.00 inches', 'is portable': 'y', 'frame material': 'nylon', 'model': 'ev0800-400', 'uv rating': '400', 'multi pack indicator': 'no', 'gender': 'men', 'brand': 'nike', 'age group': 'adult', 'lens type': 'sunglasses', 'features': 'wrap around fit; water repellent lenses; interchangeable lenses', 'material': 'plastic', 'model no.': 'ev0800-400', 'shape': 'semi-rimless', 'eyewear frame style': 'semi-rimless', 'shipping weight (in pounds)': '1.0', 'walmart no.': '554240966'

In [102]:
df['feat_age group'].value_counts()

adult               4563
men                  350
child                 77
men's                 33
unisex                 6
infant                 4
toddler                4
mens                   4
boys'                  3
youth                  2
men||women             2
women ,�� unisex       2
women                  2
12 up                  1
adult ,�� teen         1
Name: feat_age group, dtype: int64

In [101]:
  df.feat_age.value_counts()

adult                    92
modern (2000-present)    13
0+ yrs                   12
Name: feat_age, dtype: int64

In [104]:
df.feat_material.value_counts()

leather                      1843
synthetic                    1068
canvas                        743
suede                         532
polyester                     152
                             ... 
polyester/wool                  1
mesh upper with overlays        1
98% cotton/2% spandex           1
92% polyester, 8% spandex       1
soft suede upper                1
Name: feat_material, Length: 373, dtype: int64

In [0]:
'''
feats_search = ['brand_cat'
              ,'feat_shoe category_cat'
              ,'feat_style_cat'
              ,'feat_casual & dress shoe style_cat'
              ,'feat_gender_cat'
              ,'feat_brand_cat'
              ,'feat_inseam_cat'
              ,	'feat_material_cat'
              ,'feat_resizable_cat'
              ,'feat_shoe size_cat'
              ,'feat_condition_cat'
              ,'feat_weight_cat'
              ,'feat_bridge_cat'
              ,'feat_adjustable_cat'
              ,'feat_age_cat'
              ,'feat_color_cat'
              ,'feat_metal type_cat']
'''